# Simple Storm Surge Inundation Modeling

_**Caitlin Haedrich and Pratikshya Regmi**, North Carolina State University_


In this example we will use some of GRASS GIS hydrology tools, namely:

* [r.lake](https://grass.osgeo.org/grass-stable/manuals/r.lake.html): to fill a lake to a target water level from a given start point or seed raster
* [r.lake.series](https://grass.osgeo.org/grass-stable/manuals/r.lake.series.html): addon which runs r.lake for different water levels


***

## 1. Import Python Packages and Start GRASS Session

In [ ]:
# Import Python standard library and IPython packages we need.
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

For this notebook, we'll make a new mapset so that all the innundation rasters we create are organized away from the PERMANENT mapset. Our elevation rasters are in PERMANENT so we can still access them from here.

In [ ]:
# Start GRASS Session
gj.init("nags_head/PERMANENT")

# Make a new mapset for this part
gs.run_command("g.mapset", mapset="flooding", location="nags_head", flags="c")

In [ ]:
# gj.init("nags_head/flooding"); # if re-running this notebook, no need to re-create the mapset

Let's change the region to just the right half of the `jockeys_ridge` area.

In [ ]:
!g.region region=jockeys_ridge w=w+1200 -p

In [ ]:
flood1 = gj.Map(use_region=True)
flood1.d_rast(map="JR_2014")
flood1.show()

Finally, let's install the `r.lake.series` addon. GRASS GIS addons are installed with the `g.extension` tool.

In [ ]:
!g.extension extension=r.lake.series

***

## 2. Remove Nulls

`r.lake` is a bathtub model. We'll place the fill point offshore. It also won't fill null cells so our first step will be to fill the nulls. We'll do this by: 

1. Creating a mask that includes small holes but excludes the nulls offshore (r.neighbors)
2. Applying the mask and using r.fillnulls
3. Remove the mask
4. Fill the remaining, offshore nulls with a constant arbitrary value.

In [ ]:
!r.neighbors input=JR_2014_mask output=JR_2014_wide_mask method=maximum size=11
!r.mask raster=JR_2014_wide_mask
!r.fillnulls input=JR_2014 output=JR_2014_filled method=bilinear
!r.mask -r

Then, we will fill the offshore nulls with a constant value (-5 in this case).

In [ ]:
!r.mapcalc "JR_2014_filled_bath = if(isnull(JR_2014_filled), -5.0, JR_2014_filled)"

Let's see the results!

In [ ]:
!r.colors map=JR_2014_filled_bath color=elevation

flood1 = gj.Map()
flood1.d_rast(map="JR_2014_filled_bath")
flood1.show()

***

## 3. Create Fill Point 

In [ ]:
fill_point = "914151,250381"

gs.write_command("v.in.ascii", input="-", output="fill_point", separator="comma", stdin=fill_point)

In [ ]:
flood1 = gj.Map()
flood1.d_rast(map="JR_2014_filled_bath")
flood1.d_vect(map="fill_point", icon="basic/pin", size=20, fill_color="red")
flood1.show()

***

## 4. Run r.lake.series 

In [ ]:
gs.run_command("r.lake.series",
                elevation="JR_2014_filled_bath",
                output="flooding",
                start_water_level=2.0,
                end_water_level=4.0,
                water_level_step=0.2,
                coordinates=fill_point
              )

In [ ]:
from io import StringIO
import pandas as pd

pd.read_csv(StringIO(gs.read_command("t.rast.list", input="flooding", separator="comma")))

r.lake.series creates a space-time dataset. We can use the [temporal modules](https://grass.osgeo.org/grass-stable/manuals/temporal.html) to further work with the data.

***

## 5. Visualize the Results 

Let's visualize the results:

In [ ]:
flood1 = gj.Map()
flood1.d_rast(map="flooding_4.0")
flood1.show()

We can make a nicer visual.

In [ ]:
# First create a hillshade (the DEM was created with a resolution of 4 so we will compute the relief with the same)

!g.region res=4
!r.relief input=JR_2014_filled_bath output=JR_2014_shade zscale=2
!g.region res=1

In [ ]:
gs.run_command("t.rast.colors", input="flooding", color="water")

timemap = gj.TimeSeriesMap()
timemap.d_shade(shade="JR_2014_shade", color="naip_rgb")
timemap.add_raster_series("flooding")
timemap.d_legend(at=(50, 90, 80, 85))
#timemap.show() 

In [ ]:
timemap.save(filename="flood.gif");

In [ ]:
from IPython.display import display, Image

display(Image(filename="flood.gif", width=400))

## Further Readings

[Clinch, A.S., Russ, E.,Oliver, R.C., Mitasova H., and Overton, M.F. 2012, Remote sensing estimation of Hurricane Irene soundside surge elevations on the North Carolina Outer Banks, Shore and Beach 80, p. 1-10.](https://geospatial.ncsu.edu/geoforall/publications/clinch_shorepaper2012.pdf)